<a href="https://colab.research.google.com/github/SMayurgavali/CA-EXPERIMENT/blob/main/Combined_ANN_Models_with_Various_Activation_Functions_%26_Results_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
import json
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

def load_dataset(file_path: str):
    """
    Loads a dataset from a specified file path.
    Supports CSV, Excel (.xls, .xlsx), and JSON formats.
    Prints basic information about the loaded dataset.

    Args:
        file_path (str): The full path to the dataset file.

    Returns:
        pd.DataFrame: The loaded dataset as a pandas DataFrame, or None if an error occurs.
    """
    clean_file_path = file_path.strip()
    file_extension = os.path.splitext(clean_file_path)[1].lower().strip()

    try:
        if file_extension == '.csv':
            df = pd.read_csv(clean_file_path)
        elif file_extension in ['.xls', '.xlsx']:
            df = pd.read_excel(clean_file_path)
        elif file_extension == '.json':
            df = pd.read_json(clean_file_path)
        else:
            print(f"Unsupported file type: '{file_extension}'. Supported types are .csv, .xls, .xlsx, .json.")
            return None
        print("\nDataset loaded successfully!")
        print("First 5 rows of the dataset:")
        print(df.head())
        print(f"\nDataset shape: {df.shape}")
        print("\nDataset Info:")
        df.info()
        return df
    except FileNotFoundError:
        print(f"Error: File not found at '{clean_file_path}'. Please check the path and try again.")
        return None
    except pd.errors.EmptyDataError:
        print(f"Error: The file at '{clean_file_path}' is empty. Please provide a file with data.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while loading the dataset: {e}")
        return None

def sigmoid(z):
    """
    Sigmoid activation function.
    Squashes input values to a range between 0 and 1.
    """
    return 1 / (1 + np.exp(-z))

def tanh(z):
    """
    Hyperbolic Tangent (Tanh) activation function.
    Squashes input values to a range between -1 and 1.
    """
    return np.tanh(z)

def relu(z):
    """
    Rectified Linear Unit (ReLU) activation function.
    Outputs the input directly if positive, otherwise outputs 0.
    """
    return np.maximum(0, z)

def leaky_relu(z, alpha=0.01):
    """
    Leaky Rectified Linear Unit (Leaky ReLU) activation function.
    Outputs the input directly if positive, otherwise outputs a small multiple (alpha) of the input.
    Addresses the 'dying ReLU' problem.
    """
    return np.maximum(alpha * z, z)

def softmax_binary_output(z):
    """
    Softmax activation function adapted for binary classification.
    Equivalent to sigmoid for a two-class problem.
    """
    return np.exp(z) / (np.exp(z) + 1)

def get_activation_function(name: str):
    """
    Retrieves an activation function by its name.

    Args:
        name (str): The name of the activation function (e.g., 'sigmoid', 'relu').

    Returns:
        function: The corresponding activation function.

    Raises:
        ValueError: If an unsupported activation function name is provided.
    """
    activation_functions = {
        'sigmoid': sigmoid,
        'tanh': tanh,
        'relu': relu,
        'leaky_relu': leaky_relu,
        'softmax': softmax_binary_output
    }
    func = activation_functions.get(name.lower())
    if func is None:
        raise ValueError(f"Unsupported activation function '{name}'. "
                         f"Available options: {', '.join(activation_functions.keys())}.")
    return func

def train_model(X_train, y_train, activation_func, learning_rate=0.005, num_iterations=10000):
    """
    Trains a single-layer perceptron model using gradient descent.

    Args:
        X_train (np.ndarray): Training features.
        y_train (np.ndarray): Training target labels (0 or 1).
        activation_func (function): The activation function to use.
        learning_rate (float): The learning rate for gradient descent.
        num_iterations (int): The number of training iterations.

    Returns:
        tuple: A tuple containing the learned weights (np.ndarray), bias (float), and loss history.
    """
    n_samples, n_features = X_train.shape
    weights = np.zeros(n_features)
    bias = 0
    loss_history = []

    print(f"\nStarting training with {activation_func.__name__.replace('_binary_output', '')} activation...")
    print(f"Learning Rate: {learning_rate}, Iterations: {num_iterations}")

    for i in range(num_iterations):
        linear_model = np.dot(X_train, weights) + bias
        y_predicted_raw = activation_func(linear_model)

        dw = (1 / n_samples) * np.dot(X_train.T, (y_predicted_raw - y_train))
        db = (1 / n_samples) * np.sum(y_predicted_raw - y_train)

        weights -= learning_rate * dw
        bias -= learning_rate * db

        if activation_func in [sigmoid, softmax_binary_output]:
            y_predicted_clipped = np.clip(y_predicted_raw, 1e-10, 1 - 1e-10)
            current_loss = -np.mean(y_train * np.log(y_predicted_clipped) + (1 - y_train) * np.log(1 - y_predicted_clipped))
        else:
            current_loss = np.mean((y_predicted_raw - y_train)**2)

        loss_history.append(current_loss)

        if (i + 1) % (num_iterations // 10) == 0:
            print(f"Iteration {i+1}/{num_iterations}, Loss: {current_loss:.6f}")

    print("Training complete.")
    return weights, bias, loss_history

def predict_classes(X, weights, bias, activation_func, threshold=0.5):
    """
    Makes binary class predictions using the trained model.
    """
    linear_model = np.dot(X, weights) + bias
    y_predicted_raw = activation_func(linear_model)
    y_predicted_classes = (y_predicted_raw >= threshold).astype(int)
    return y_predicted_classes

def evaluate_model(y_true, y_pred, model_name="Model"):
    """
    Evaluates the model's performance using various classification metrics.
    """
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    cm = confusion_matrix(y_true, y_pred)

    print(f"\n--- Evaluation Metrics for {model_name} ---")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print(f"  True Negative (TN): {cm[0,0]}")
    print(f"  False Positive (FP): {cm[0,1]}")
    print(f"  False Negative (FN): {cm[1,0]}")
    print(f"  True Positive (TP): {cm[1,1]}")
    return accuracy, precision, recall, f1, cm

def save_model(weights, bias, activation_name, scaler, file_path="model_params.pkl"):
    """Saves the trained model parameters and scaler to a file."""
    model_data = {
        'weights': weights,
        'bias': bias,
        'activation_name': activation_name,
        'scaler': scaler
    }
    with open(file_path, 'wb') as f:
        pickle.dump(model_data, f)
    print(f"\nModel parameters saved successfully to '{file_path}'")

def load_model(file_path="model_params.pkl"):
    """Loads a trained model from a file."""
    try:
        with open(file_path, 'rb') as f:
            model_data = pickle.load(f)
        print(f"\nModel parameters loaded successfully from '{file_path}'")
        return model_data['weights'], model_data['bias'], model_data['activation_name'], model_data['scaler']
    except FileNotFoundError:
        print(f"Error: Model file not found at '{file_path}'.")
        return None, None, None, None
    except Exception as e:
        print(f"An error occurred while loading the model: {e}")
        return None, None, None, None


if __name__ == "__main__":
    print("🚀 Advanced Perceptron Model for Binary Classification")
    print("This script trains and evaluates a single-layer perceptron with multiple activation functions.")
    print("It includes data preprocessing, evaluation metrics, and model saving/loading.")

    dataset_path = input("Enter the full path to your dataset file: ").strip()

    if not dataset_path:
        print("No path entered. Exiting.")
    else:
        my_data = load_dataset(dataset_path)

        if my_data is not None:
            try:
                # Basic data cleaning: drop rows with any missing values
                my_data = my_data.dropna()

                # Separate features (X) and target (y)
                X = my_data.iloc[:, :-1].select_dtypes(include=np.number).values
                y = my_data.iloc[:, -1].astype(int).values

                if not np.all(np.isin(y, [0, 1])):
                    print("Error: The target variable (last column) is not binary (0 or 1). Exiting.")
                    exit()

                # Feature Scaling
                scaler = StandardScaler()
                X_scaled = scaler.fit_transform(X)

                # Split data into training and testing sets
                X_train, X_test, y_train, y_test = train_test_split(
                    X_scaled, y, test_size=0.2, random_state=42, stratify=y
                )
                print(f"\nData split into training ({len(X_train)} samples) and testing ({len(X_test)} samples).")

                print("\nAvailable Activations: Sigmoid, Tanh, ReLU, Leaky_ReLU, Softmax")
                activation_choices_str = input("Enter desired activation functions, separated by commas: ").strip()
                selected_activations = [choice.strip() for choice in activation_choices_str.split(',') if choice.strip()]

                if not selected_activations:
                    print("No activation functions selected. Exiting.")
                    exit()

                try:
                    user_learning_rate = float(input("Enter learning rate (e.g., 0.005): ") or "0.005")
                    user_num_iterations = int(input("Enter number of iterations (e.g., 10000): ") or "10000")
                except ValueError:
                    print("Invalid input. Using defaults (0.005, 10000).")
                    user_learning_rate = 0.005
                    user_num_iterations = 10000

                results = {}
                best_activation = None
                highest_accuracy = -1.0

                for act_name in selected_activations:
                    try:
                        chosen_activation_func = get_activation_function(act_name)
                        print(f"\n--- Running model with {act_name} activation ---")

                        weights, bias, loss_history = train_model(
                            X_train, y_train, chosen_activation_func,
                            learning_rate=user_learning_rate,
                            num_iterations=user_num_iterations
                        )

                        # Plotting the loss history
                        plt.figure()
                        plt.plot(loss_history)
                        plt.title(f'Loss History for {act_name} Activation')
                        plt.xlabel('Iterations')
                        plt.ylabel('Loss')
                        plt.grid(True)
                        plt.show()

                        y_pred_test = predict_classes(X_test, weights, bias, chosen_activation_func)
                        accuracy, precision, recall, f1, cm = evaluate_model(y_test, y_pred_test, model_name=f"{act_name} (Test Set)")

                        results[act_name] = {
                            "weights": weights, "bias": bias, "test_accuracy": accuracy,
                            "test_precision": precision, "test_recall": recall, "test_f1": f1,
                            "test_confusion_matrix": cm
                        }

                        if accuracy > highest_accuracy:
                            highest_accuracy = accuracy
                            best_activation = act_name
                            best_weights = weights
                            best_bias = bias

                    except ValueError as ve:
                        print(f"Error during training with {act_name}: {ve}")
                    except Exception as e:
                        print(f"An unexpected error occurred during training with {act_name}: {e}")


                print("\n--- Model Training Summary ---")
                for act_name, metrics in results.items():
                    print(f"\n{act_name} Activation:")
                    print(f"  Test Accuracy: {metrics['test_accuracy']:.2f}%")
                    print(f"  Test Precision: {metrics['test_precision']:.4f}")
                    print(f"  Test Recall: {metrics['test_recall']:.4f}")
                    print(f"  Test F1-Score: {metrics['test_f1']:.4f}")

                if best_activation:
                    print(f"\nBest performing activation on test set: {best_activation} (Accuracy: {highest_accuracy:.2f}%)")

                    save_model(best_weights, best_bias, best_activation, scaler)

                    # Example of loading and using the saved model
                    loaded_weights, loaded_bias, loaded_activation_name, loaded_scaler = load_model()

                    if loaded_weights is not None:
                        loaded_activation_func = get_activation_function(loaded_activation_name)
                        # Assuming you want to predict on the test set again with the loaded model
                        X_test_scaled = loaded_scaler.transform(my_data.iloc[:, :-1].select_dtypes(include=np.number).values) # Use original data for scaling
                        y_pred_loaded = predict_classes(X_test_scaled, loaded_weights, loaded_bias, loaded_activation_func)
                        print("\nEvaluation of the Loaded Model:")
                        evaluate_model(my_data.iloc[:, -1].astype(int).values, y_pred_loaded, model_name="Loaded Model (Full Data)") # Evaluate on full data to match scaling


            except Exception as e:
                print(f"An error occurred during data processing or model execution: {e}")

🚀 Advanced Perceptron Model for Binary Classification
This script trains and evaluates a single-layer perceptron with multiple activation functions.
It includes data preprocessing, evaluation metrics, and model saving/loading.
Enter the full path to your dataset file: /content/data.csv

Dataset loaded successfully!
First 5 rows of the dataset:
   feature_A,feature_B,target_label  Unnamed: 1  Unnamed: 2
0                                 6          20           1
1                                 4          23           0
2                                 5          21           1
3                                 2          26           1
4                                 3          28           0

Dataset shape: (6, 3)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column                            Non-Null Count  Dtype
---  ------                            --------------  -----
 0   feature_A,feature_B,target_la